<a href="https://colab.research.google.com/github/bob8dod/ML-studying/blob/main/2021ML/8.%20Pipeline%2CCrossValidation%2CGridSearchCV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Piepline _ 내가 설정한 모델 만들기

In [ ]:
from sklearn.datasets import load_breast_cancer
cancer = load_breast_cancer()
X = cancer.data
Y = cancer.target

from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2, stratify=Y, random_state=1)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

from sklearn.pipeline import make_pipeline
pipe = make_pipeline(StandardScaler(), PCA(n_components=4), LogisticRegression())
pipe.fit(X_train,Y_train)
pipe.score(X_test, Y_test) #이게 장점임, 원래는 test data에다가 일일히 다 전처리해줘야되는데 걍 pipe에 넣어주면 끝

0.9736842105263158

In [ ]:
pipe.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'standardscaler', 'pca', 'logisticregression', 'standardscaler__copy', 'standardscaler__with_mean', 'standardscaler__with_std', 'pca__copy', 'pca__iterated_power', 'pca__n_components', 'pca__random_state', 'pca__svd_solver', 'pca__tol', 'pca__whiten', 'logisticregression__C', 'logisticregression__class_weight', 'logisticregression__dual', 'logisticregression__fit_intercept', 'logisticregression__intercept_scaling', 'logisticregression__l1_ratio', 'logisticregression__max_iter', 'logisticregression__multi_class', 'logisticregression__n_jobs', 'logisticregression__penalty', 'logisticregression__random_state', 'logisticregression__solver', 'logisticregression__tol', 'logisticregression__verbose', 'logisticregression__warm_start'])

# 교차검증

In [ ]:
import numpy as np
from sklearn.model_selection import cross_validate
scores = cross_validate(pipe, X_train, Y_train, cv=10, return_train_score=True) #k값, k개의 다른 검증데이터를 통해서 검증
  /#return_train_score=True, default는 False 학습 과정에서 일어난 학습데이터의 accuracy를 return해주는 기능 -> 각각의 교차검증에서 일어나는 score를 확인할 수 있음 -> scores['train_score'] / score['test_score']
print(scores.keys())

dict_keys(['fit_time', 'score_time', 'test_score', 'train_score'])


In [ ]:
print('CV Train Accuracy scores: ', scores['train_score'])
print('CV Train Accuracy: %.3f +/- %.3f' %(np.mean(scores['train_score']), np.std(scores['train_score'])))

CV Train Accuracy scores:  [0.96577017 0.96577017 0.96577017 0.96577017 0.96821516 0.96585366
 0.97073171 0.96829268 0.97560976 0.96585366]
CV Train Accuracy: 0.968 +/- 0.003


In [ ]:
print('CV Validation Accuracy scores: ', scores['test_score'])
print('CV Validation Accuracy: %.3f +/- %.3f' %(np.mean(scores['test_score']), np.std(scores['test_score'])))

CV Validation Accuracy scores:  [0.97826087 0.97826087 0.95652174 1.         0.95652174 0.97777778
 0.93333333 0.95555556 0.91111111 1.        ]
CV Validation Accuracy: 0.965 +/- 0.027


### 교차 검증 결과
- 과대적합도 과소적합도 발생하지 않는다

# 최적의 모델 선정1(GridSearchCV + make_pipeline) _ 기초



In [ ]:
from sklearn.model_selection import GridSearchCV
param = {}
clf = GridSearchCV(pipe, param,scoring='accuracy',cv=10) #여기서의 cv가 cross_validation : 교차검증
clf.fit(X_train, Y_train)
print(clf.cv_results_) #교차검증 결과를 확인
best = clf.best_estimator_ #가장 좋은 성능을 보인 모델을 선정!!
print(clf.score(X_train,Y_train)) #바로 score하거나 predict해도 best모델로 해줌
print(best.score(X_train,Y_train)) 
print(best.score(X_test,Y_test))

{'mean_fit_time': array([0.0096966]), 'std_fit_time': array([0.0030157]), 'mean_score_time': array([0.00078752]), 'std_score_time': array([3.18392726e-05]), 'params': [{}], 'split0_test_score': array([0.97826087]), 'split1_test_score': array([0.97826087]), 'split2_test_score': array([0.95652174]), 'split3_test_score': array([1.]), 'split4_test_score': array([0.95652174]), 'split5_test_score': array([0.97777778]), 'split6_test_score': array([0.93333333]), 'split7_test_score': array([0.95555556]), 'split8_test_score': array([0.91111111]), 'split9_test_score': array([1.]), 'mean_test_score': array([0.9647343]), 'std_test_score': array([0.02665336]), 'rank_test_score': array([1], dtype=int32)}
0.967032967032967
0.967032967032967
0.9736842105263158


# 최적의 모델 선정(GridSearchCV + Pipline, ParameterGrid) _ 심화

In [ ]:
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

In [ ]:
# 파라미터 Parsing
from sklearn.pipeline import Pipeline
estimators = [('normalization', StandardScaler()), ('clf', SVC())]
pipeline = Pipeline(estimators)

In [ ]:
#Parmeter_grid 생성해주기
from sklearn.model_selection import ParameterGrid 
grid = [{'clf__kernel': [['linear']], 'clf__C': [[0.001], [0.01],[0.1],[1],[10], [100], [1000]]}, 
        {'clf__kernel': [['rbf']], 'clf__gamma': [[0.001], [0.01],[0.1],[1],[10], [100], [1000]], 'clf__C': [[0.001], [0.01],[0.1],[1],[10], [100], [1000]]}]
# 언더바2개 __ 를 통해서 해당 속성과 연결해줌, 
grid_param = ParameterGrid(grid)
list(grid_param) #설정한 파라미터를 통해서 나오는 모든 경우의 수를 보여줌

[{'clf__C': [0.001], 'clf__kernel': ['linear']},
 {'clf__C': [0.01], 'clf__kernel': ['linear']},
 {'clf__C': [0.1], 'clf__kernel': ['linear']},
 {'clf__C': [1], 'clf__kernel': ['linear']},
 {'clf__C': [10], 'clf__kernel': ['linear']},
 {'clf__C': [100], 'clf__kernel': ['linear']},
 {'clf__C': [1000], 'clf__kernel': ['linear']},
 {'clf__C': [0.001], 'clf__gamma': [0.001], 'clf__kernel': ['rbf']},
 {'clf__C': [0.001], 'clf__gamma': [0.01], 'clf__kernel': ['rbf']},
 {'clf__C': [0.001], 'clf__gamma': [0.1], 'clf__kernel': ['rbf']},
 {'clf__C': [0.001], 'clf__gamma': [1], 'clf__kernel': ['rbf']},
 {'clf__C': [0.001], 'clf__gamma': [10], 'clf__kernel': ['rbf']},
 {'clf__C': [0.001], 'clf__gamma': [100], 'clf__kernel': ['rbf']},
 {'clf__C': [0.001], 'clf__gamma': [1000], 'clf__kernel': ['rbf']},
 {'clf__C': [0.01], 'clf__gamma': [0.001], 'clf__kernel': ['rbf']},
 {'clf__C': [0.01], 'clf__gamma': [0.01], 'clf__kernel': ['rbf']},
 {'clf__C': [0.01], 'clf__gamma': [0.1], 'clf__kernel': ['rbf']},

In [ ]:
gs = GridSearchCV(pipeline, grid_param, scoring='accuracy', cv=10, n_jobs=1)
gs.fit(X_train, Y_train)
print(gs.score(X_train, Y_train))
print(gs.score(X_test, Y_test))

0.978021978021978
0.9824561403508771


In [ ]:
gs.best_score_ #??왜 score이랑 best score이랑 차이나는지 모르겠음

0.9758454106280192

In [ ]:
gs.best_params_

{'clf__C': 10, 'clf__gamma': 0.001, 'clf__kernel': 'rbf'}